In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p2/X_test.csv
/kaggle/input/2021-ai-w6-p2/train.csv
/kaggle/input/2021-ai-w6-p2/sample_submit.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)

In [4]:
import random
random.seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [5]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w6-p2/train.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w6-p2/X_test.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w6-p2/sample_submit.csv")
print(train_data.shape)
print(test_data.shape)
print(submit_data.shape)

(9999, 9)
(2481, 9)
(2481, 2)


In [6]:
print(train_data.max())
print(test_data.max())
print(train_data.min())
print(test_data.min())

Day        208.000000
Hour        23.000000
Minute      30.000000
DHI        528.000000
DNI       1059.000000
WS           8.000000
RH         100.000000
T           33.000000
TARGET      97.849989
dtype: float64
ID        2480.00
Day       1094.00
Hour        23.00
Minute      30.00
DHI        242.00
DNI       1023.00
WS           8.00
RH          96.09
T           20.00
dtype: float64
Day        0.00
Hour       0.00
Minute     0.00
DHI        0.00
DNI        0.00
WS         0.10
RH        13.42
T        -14.00
TARGET     0.00
dtype: float64
ID           0.00
Day       1043.00
Hour         0.00
Minute       0.00
DHI          0.00
DNI          0.00
WS           0.00
RH          15.22
T          -11.00
dtype: float64


In [7]:
xtrain=torch.FloatTensor(train_data.iloc[:, :-1].values).to(device)
ytrain=torch.FloatTensor(train_data['TARGET'].values).to(device)
print(xtrain)
print(ytrain)

tensor([[  0.0000,   0.0000,   0.0000,  ...,   1.5000,  69.0800, -12.0000],
        [  0.0000,   0.0000,  30.0000,  ...,   1.5000,  69.0600, -12.0000],
        [  0.0000,   1.0000,   0.0000,  ...,   1.6000,  71.7800, -12.0000],
        ...,
        [208.0000,   6.0000,   0.0000,  ...,   1.1000,  47.4600,  15.0000],
        [208.0000,   6.0000,  30.0000,  ...,   1.4000,  44.5100,  17.0000],
        [208.0000,   7.0000,   0.0000,  ...,   1.7000,  37.8000,  19.0000]],
       device='cuda:0')
tensor([ 0.0000,  0.0000,  0.0000,  ..., 13.8872, 23.2699, 33.0276],
       device='cuda:0')


In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(xtrain.cpu())
xtrain2=scaler.transform(xtrain.cpu())
xtrain2=torch.FloatTensor(xtrain2).to(device)
print(xtrain2)

tensor([[-1.7237, -1.6589, -0.9999,  ..., -0.6411,  0.2243, -2.1140],
        [-1.7237, -1.6589,  1.0001,  ..., -0.6411,  0.2234, -2.1140],
        [-1.7237, -1.5145, -0.9999,  ..., -0.5616,  0.3471, -2.1140],
        ...,
        [ 1.7352, -0.7925, -0.9999,  ..., -0.9593, -0.7593,  0.7033],
        [ 1.7352, -0.7925,  1.0001,  ..., -0.7207, -0.8935,  0.9120],
        [ 1.7352, -0.6481, -0.9999,  ..., -0.4820, -1.1987,  1.1207]],
       device='cuda:0')


In [9]:
# print(xtrain2)

In [10]:
linear1=torch.nn.Linear(8,16, bias=True)
linear2=torch.nn.Linear(16,16, bias=True)
linear3=torch.nn.Linear(16,8, bias=True)
linear4=torch.nn.Linear(8,1, bias=True)
sigmoid=torch.nn.Sigmoid()

In [11]:
model=torch.nn.Sequential(linear1,sigmoid,linear2,sigmoid,linear3,sigmoid,linear4).to(device)
model

Sequential(
  (0): Linear(in_features=8, out_features=16, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=16, out_features=8, bias=True)
  (5): Sigmoid()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

In [12]:
print(xtrain2)
ytrain2=ytrain.unsqueeze(1)
print(ytrain2)

tensor([[-1.7237, -1.6589, -0.9999,  ..., -0.6411,  0.2243, -2.1140],
        [-1.7237, -1.6589,  1.0001,  ..., -0.6411,  0.2234, -2.1140],
        [-1.7237, -1.5145, -0.9999,  ..., -0.5616,  0.3471, -2.1140],
        ...,
        [ 1.7352, -0.7925, -0.9999,  ..., -0.9593, -0.7593,  0.7033],
        [ 1.7352, -0.7925,  1.0001,  ..., -0.7207, -0.8935,  0.9120],
        [ 1.7352, -0.6481, -0.9999,  ..., -0.4820, -1.1987,  1.1207]],
       device='cuda:0')
tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        ...,
        [13.8872],
        [23.2699],
        [33.0276]], device='cuda:0')


In [13]:
loss=torch.nn.MSELoss().to(device)
optimizer=optim.SGD(model.parameters(),lr=1e-2)

epochs=10000

for epoch in range(epochs+1):
    optimizer.zero_grad()
    hypothesis=model(xtrain2)
    cost=loss(hypothesis, ytrain2)
    cost.backward()
    optimizer.step()
    if epoch%1000==0:
        print('epoch {} , cost {:.9f}'.format(epoch,cost.item()))

epoch 0 , cost 1035.615966797
epoch 1000 , cost 6.998241901
epoch 2000 , cost 3.113094091
epoch 3000 , cost 1.789397717
epoch 4000 , cost 1.256005526
epoch 5000 , cost 0.986514866
epoch 6000 , cost 0.819902003
epoch 7000 , cost 0.706208110
epoch 8000 , cost 0.625753343
epoch 9000 , cost 0.566712141
epoch 10000 , cost 0.521335125


In [14]:
hypothesis=model(xtrain2)
print(hypothesis)
print(ytrain2)

tensor([[ 0.3739],
        [ 0.3635],
        [ 0.3391],
        ...,
        [14.7681],
        [25.0837],
        [35.1383]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        ...,
        [13.8872],
        [23.2699],
        [33.0276]], device='cuda:0')


In [15]:
print(test_data.iloc[:, 1:])
tdata=torch.FloatTensor(scaler.transform(test_data.iloc[:, 1:].values)).to(device)
print(tdata)

       Day  Hour  Minute  DHI  DNI   WS     RH   T
0     1043     7      30   34  584  2.0  63.02   3
1     1043     8       0   45  730  2.3  61.54   5
2     1043     8      30   52  817  2.4  57.34   6
3     1043     9       0   57  878  2.6  52.46   8
4     1043     9      30   61  917  2.2  45.74  10
...    ...   ...     ...  ...  ...  ...    ...  ..
2476  1094    21      30    0    0  2.4  70.70  -4
2477  1094    22       0    0    0  2.4  66.79  -4
2478  1094    22      30    0    0  2.2  66.78  -4
2479  1094    23       0    0    0  2.1  67.72  -4
2480  1094    23      30    0    0  2.1  67.70  -4

[2481 rows x 8 columns]
tensor([[ 1.5621e+01, -6.4805e-01,  1.0001e+00,  ..., -2.4337e-01,
         -5.1398e-02, -5.4879e-01],
        [ 1.5621e+01, -5.0364e-01, -9.9990e-01,  ..., -4.7257e-03,
         -1.1873e-01, -3.4010e-01],
        [ 1.5621e+01, -5.0364e-01,  1.0001e+00,  ...,  7.4823e-02,
         -3.0980e-01, -2.3576e-01],
        ...,
        [ 1.6469e+01,  1.5181e+00,  1.000

In [16]:
hypothesis=model(tdata)
print(hypothesis)

tensor([[22.4530],
        [27.4821],
        [31.3280],
        ...,
        [ 5.8268],
        [ 5.8494],
        [ 5.8262]], device='cuda:0', grad_fn=<AddmmBackward>)


In [17]:
submit_data['TARGET']=hypothesis.cpu().detach().numpy()
print(submit_data)

        ID     TARGET
0        0  22.453024
1        1  27.482071
2        2  31.327969
3        3  33.733025
4        4  36.328392
...    ...        ...
2476  2476   5.924610
2477  2477   5.879372
2478  2478   5.826769
2479  2479   5.849434
2480  2480   5.826155

[2481 rows x 2 columns]


In [18]:
submit_data.to_csv('submission.csv',index=False,mode='w')